In [33]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
from citipy import citipy
from datetime import datetime
from scipy.stats import linregress

# Import the requests library
import requests

# Import the API key
from config import weather_api_key
from config import g_key
gmaps.configure(api_key=g_key)

In [34]:
# import damp_cities from Part 1
cities_to_load = "weather_data/M6_data/WeatherPy_challenge.csv"

In [35]:
# Create new dataframe from WeatherPy_challenge.csv
filtered_cities_data_df = pd.read_csv(cities_to_load)
filtered_cities_data_df

,Weather_Challenge,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Cape Town,ZA,2019-11-11 07:06:56,-33.93,18.42,66.00,82,90,6.93,overcast clouds,0.00,0.0
1,1,Rikitea,PF,2019-11-11 07:06:56,-23.12,-134.97,73.99,85,93,20.33,light rain,0.63,0.0
2,2,Ushuaia,AR,2019-11-11 07:06:46,-54.81,-68.31,50.00,61,75,4.70,broken clouds,0.00,0.0
3,3,New Norfolk,AU,2019-11-11 07:06:57,-42.78,147.06,73.40,38,85,24.16,overcast clouds,0.00,0.0
4,4,Kaeo,NZ,2019-11-11 07:06:57,-35.10,173.78,55.00,100,100,19.55,light rain,1.94,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,555,Pokrovsk,RU,2019-11-11 07:10:13,61.48,129.14,-1.54,83,99,6.78,overcast clouds,0.00,0.0
556,556,Novikovo,RU,2019-11-11 07:10:14,51.15,37.89,41.11,90,3,12.10,clear sky,0.00,0.0
557,557,Muros,ES,2019-11-11 07:10:14,42.77,-9.06,55.00,100,75,2.24,light intensity drizzle,0.00,0.0
558,558,Humen,CN,2019-11-11 07:10:15,22.82,113.67,84.99,51,20,8.05,few clouds,0.00,0.0


In [36]:
# determine df types
filtered_cities_data_df.dtypes

Weather_Challenge             int64
City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain inches (last 3 hrs)    float64
Snow inches (last 3 hrs)    float64
dtype: object

In [37]:
# Determine user-inputted temp preferences

# Ask the customer to add a minimum and maximum temperature value.
min_trip_temp = float(input("What is your preferred minimum temperature? "))
max_trip_temp = float(input("What is your preferred maximum temperature? "))

What is your preferred minimum temperature? 20
What is your preferred maximum temperature? 70


In [38]:
# Determine user inputted precip preferences

# Ask the customer to add a minimum and maximum temperature value.
rain_pref = str(input("Do you want it to be raining? (yes/no)? "))
snow_pref = str(input("Do you want it to be snowing? (yes/no)? "))

Do you want it to be raining? (yes/no)? no
Do you want it to be snowing? (yes/no)? no


In [39]:
# Filter the dataset to find the cities that fit the TEMPERATURE, RAIN and SNOW preferences  

filtered_cities_data_df = filtered_cities_data_df.loc[(filtered_cities_data_df["Max Temp"] <= max_trip_temp) & \
                                       (filtered_cities_data_df["Max Temp"] >= min_trip_temp) & \
                                       (np.where(filtered_cities_data_df["Rain inches (last 3 hrs)"] > 0, "yes","no") == rain_pref) & \
                                       (np.where(filtered_cities_data_df["Snow inches (last 3 hrs)"] > 0, "yes","no") == snow_pref)]

filtered_cities_data_df.head()

,Weather_Challenge,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Cape Town,ZA,2019-11-11 07:06:56,-33.93,18.42,66.0,82,90,6.93,overcast clouds,0.0,0.0
2,2,Ushuaia,AR,2019-11-11 07:06:46,-54.81,-68.31,50.0,61,75,4.70,broken clouds,0.0,0.0
8,8,East London,ZA,2019-11-11 07:06:58,-33.02,27.91,64.4,93,90,14.99,light intensity drizzle,0.0,0.0
9,9,Albany,US,2019-11-11 07:06:58,42.65,-73.75,41.0,64,90,1.48,overcast clouds,0.0,0.0
10,10,Wakkanai,JP,2019-11-11 07:06:59,45.41,141.67,48.2,76,75,13.87,broken clouds,0.0,0.0


In [40]:
# reorder the columns

new_vacation_column_order = ["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]


In [41]:
# Reordered DataFrame
filtered_cities_data_df = filtered_cities_data_df[new_vacation_column_order]
filtered_cities_data_df["Hotel Name"] = ""
filtered_cities_data_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,66.0,overcast clouds,-33.93,18.42,
2,Ushuaia,AR,50.0,broken clouds,-54.81,-68.31,
8,East London,ZA,64.4,light intensity drizzle,-33.02,27.91,
9,Albany,US,41.0,overcast clouds,42.65,-73.75,
10,Wakkanai,JP,48.2,broken clouds,45.41,141.67,


In [42]:
# Set parameters to search for a hotel
params = {"radius": 5000, 
          "type": "lodging", 
          "key": g_key}

In [43]:
# Iterate through the DataFrame - REMEMBER TO CHANGE FILTERED_SNOW_DATA
for index, row in filtered_cities_data_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]    
    lng = row["Lng"]
    
    # Add the latitude and longitude to the params dictionary as values to the location key
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name
    try:
        filtered_cities_data_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError)    :
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [44]:
filtered_cities_data_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,66.00,overcast clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
2,Ushuaia,AR,50.00,broken clouds,-54.81,-68.31,Albatros Hotel
8,East London,ZA,64.40,light intensity drizzle,-33.02,27.91,Tu Casa
9,Albany,US,41.00,overcast clouds,42.65,-73.75,Hilton Albany
10,Wakkanai,JP,48.20,broken clouds,45.41,141.67,Hyōsetsu-sō
16,Ponta Do Sol,BR,68.20,broken clouds,-20.63,-46.00,Hotel Escarpas do Lago
20,Bredasdorp,ZA,69.01,overcast clouds,-34.53,20.04,Bredasdorp Country Manor
21,Aasiaat,GL,37.83,overcast clouds,68.71,-52.87,Hotel Aasiaat Sømandshjem
22,Ziezmariai,LT,43.00,mist,54.80,24.45,Dalios kaimo sodyba
24,Punta Arenas,CL,46.40,broken clouds,-53.16,-70.91,Hotel Dreams Del Estrecho


In [45]:
# Create hotel box template

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row)for index, row in filtered_cities_data_df.iterrows()]

In [46]:
# Add a pop-up marker for each city
locations = filtered_cities_data_df[["Lat", "Lng"]]
#max_temp = filtered_cities_data_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
#heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
# Create the output file (CSV)
output_data_file = "weather_data/M6_data/WeatherPy_vacation.csv"

# Export the City_Data into a CSV
filtered_cities_data_df.to_csv(output_data_file, index_label="Weather_Challenge_p2")

In [48]:
# Take a screenshot of the marker layer map for the route and save it as WeatherPy_vacation_map.png.
# might not be feasible: https://jupyter-gmaps.readthedocs.io/en/latest/export.html

plt.savefig("weather_data/M6_images/WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>

### Placeholder cells below

In [49]:
# Take a screenshot of the marker layer map for the route and save it as WeatherPy_vacation_map.png.
# might not be feasible: https://jupyter-gmaps.readthedocs.io/en/latest/export.html

plt.savefig("weather_data/WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>